# DoE with classical designs

This is a template notebook for classical designs of experiments.

Author: {{ cookiecutter.author_name }}
Created: {{ cookiecutter.timestamp }}

## How to use the notebook

The following cells:
- specify the parameter search space, the objective, and any other metrics,
- specifiy, which classical design should be employed, and
- computes the list of suggested parameter sets, with which to run trials in your experiment. 

By default, the notebook is set up to run with an example. To see how it works, run the notebook (multiple times) without changing the code.

For your project, adjust the code in the linked cells with your objectives, variables, dataset etc. and then execute all cells in order.

Please refer to classical_doe.board for detailed instructions.

In [0]:
# Link to project experiments folder hypothesis_experiment_learnings.board (refresh and hit enter on this line to see the link)

## Imports  and general setup

In [0]:
import os

from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from classical_designs import get_design
from classical_designs import get_d_utility_for_polynomial_model


plt.style.use("dark_background")


## Project

In [0]:
experiment_name = '{{cookiecutter.use_case_name}}'  # please provide a name for the experiment
data_dir = "./"           # please provide a name for saving the trial data for the experiment

## Metrics

In [0]:
metrics = ["cost", "quality"]   # please provide a list of metrics

## Parameters

In [0]:
parameters = [
    # please insert the information on the names and bound/values of the parameters to try:
    {
        "name": "x1",           # the name of the parameter
        "type": "range",        # the type of parameter: "range" is for continuous parameters
        "bounds": [0., 1.],     # the lower and upper bound of the parameter as a tuple for range parameters
        "n_values": 3,          # the number of different values for the range parameter to try
    },
    {
        "name": "x2",
        "type": "range",
        "bounds": [0., 10.],
        "n_values": 3, 
    },  
    {
        "name": "x3",
        "type": "range",
        "bounds": [-5., 5.],
        "n_values": 3, 
    },  
#    {
#        "name": "x4",
#        "type": "choice",                    # the type of parameter: "choice" is for discrete parameters
#        "values": ["up", "down", "stange"],  # the values to try from for parameter#
#        "is_ordered": False,                 # whether values are ordered
#    },   
]


## Design

In [0]:
design_type = "full_factorial"               # the type of design, available are:
                                             #  - "simple_central": just a central design
                                             #  - "mixed_central": central + mixed second order design
                                             #  - "full_central": central + full 2-level factorial design
                                             #  - "full_factorial": full factorial design
                                             #  - "random_factorial": random factorial design
                                             #  - "random": random design

n_trials = 100                               # number of trials unless determined by design
sort_values = False                          # whether to sort trials by parameter values
sort_ascending = True                        # whether to sort in ascending order


## Expected performance for polynomial models

In case, all parameters are continuous, you can compute the expected performance of the design for estimating the parameters of a polynomial model.

In [0]:
order = 2                                   # the order of the polynomial model
mixed = False                               # whether to include mixed terms for order >= 2


## Trial generation

### Get the design

In [0]:
result_columns = [metric + suffix for metric in metrics for suffix in ("_mean", "_SEM")]

design = get_design(parameters=parameters,
                    design_type=design_type, 
                    n_trials=n_trials,
                    sort_values=sort_values,
                    sort_ascending=sort_ascending,
                    metrics=result_columns)

print("design:")
display(design)
print(f"type = {design_type}")
print(f"n_trials = {len(design)}")

data_file_name = os.path.join(data_dir,  f"data_{experiment_name}_running_trials.csv")
print(f"the data will be stored in: {data_file_name}")

if os.path.exists(data_file_name):
    dt = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
    os.rename(data_file_name, os.path.join(data_dir,  f"data_{experiment_name}_running_trials_{dt}.csv"))

design.to_csv(data_file_name)


### Judge the design

In [0]:
if all(p["type"] == "range" for p in parameters):
    random_design = get_design(parameters, design_type="random", n_trials=len(design), metrics=metrics)
    d_utility = get_d_utility_for_polynomial_model(parameters, design, order=order, mixed=mixed)
    random_d_utility = get_d_utility_for_polynomial_model(parameters, random_design, order=order, mixed=mixed)

    print(f"d utility for a {'mixed' if mixed else 'simple'} polynomial model of order = {order}:")
    print(f"for this {design_type} design: {d_utility}")
    print(f"for a random design: {random_d_utility}")
